# Final Project

# Title idea: What's in a Game? (reference to Shakespeare soliloquy: "What's in a name?")

For reference here is the Data Science pipeline:

Data Collection -> Data Processing -> Exploratory analysis + data visualization -> Analysis, hypothesis testing, & ML -> Insight & Policy Decision

Eventually delete this box

## Introduction

Steam is the largest digital video game distribution service available to Windows, Mac, and Linux users. Today, Steam has a collection of over 60,000 games available to consumers worldwide. Steam users are able to give either a positive or negative review for each of their owned games, and these reviews are able to be seen by other potential customers to help inform their purchasing decision. Logically, we can conclude that games with a higher positive review percentage are regarded as "better" games, in the sense that they are publicly deemed to be worthy of buying. In this tutorial we will analyze various aspects of games on Steam to try and answer the question:

**What makes a game good?**

## Data Collection

First we gather our Steam Web API key a locally stored text document. A key can be requested from https://steamcommunity.com/dev/apikey.

In [1]:
key = open('key.txt').read()

Next we'll import a few key libraries that we will make use of during data collection.

In [23]:
import requests
import pandas as pd
import numpy as np
import time

Steam has a couple of APIs that can be used to find all sorts of information about a game, including pricing information, genre, number of positive reviews, and much more. But to get this information about a game, first you need to know its designated app ID, as given by Steam. Fortunately, there is a third API provided by Steam that can be used to get a list of app IDs for every single app available on the Steam web store. The term "app" includes games, downloadable content (DLCs), software, videos, and hardware. We are only interested in the app IDs for games, so we will set our API request parameters accordingly. Additionally, only 50,000 results are allowed per request, so we will have to make this request multiple times.

In [20]:
all_ids = []
last_appid = 0

while True:
    URL = "https://api.steampowered.com/IStoreService/GetAppList/v1/"
    PARAMS = {"key": key, "max_results": 50000, "include_games": True, "include_dlc": False, "include_software": False, "include_videos": False, "include_hardware": False, "last_appid": last_appid}
    re = requests.get(url=URL, params=PARAMS).json()["response"]
    for app in re["apps"]:
        all_ids.append(app["appid"])
        
    if "last_appid" not in re.keys():
        break
    else:
        last_appid = re["last_appid"]

67320

With the list of app IDs for every game on the Steam store, we can now make calls to the previously mentioned Steam APIs to get information about every game on the Steam web store. First, we will define a list of the categories of information that we will retrieve from the API response for each game. We also create a list of filters that will be sent as a parameter for the API request.

In [21]:
categories = [
    ['name'],
    ['type'],
    ['steam_appid'],
    ['developers'],
    ['publishers'],
    ['is_free'],
    ['price_overview', 'initial'],
    ['achievements', 'total'],
    ['release_date', 'date'],
    ['metacritic', 'score']
]

filters = ''
for cat in categories:
    filters += cat[0] + ','
filters += 'basic,genres,categories'

Next we define a function to extract this information from each response, which relies on the fact that the API responses are in the JSON format.

In [22]:
def fetch(game_info, categories):
    info = {}
    for category in categories:
        cur_info = game_info
        failed = False
        
        for subpart in category:
            if not failed and subpart in cur_info.keys():
                cur_info = cur_info[subpart]
            else:
                failed = True
        
        if not failed:
            info[' '.join(category)] = cur_info
        else:
            info[' '.join(category)] = np.NaN
    
    if 'genres' in game_info.keys():
        for genre in game_info['genres']:
            info['Genre: ' + genre['description']] = True
    if 'categories' in game_info.keys():
        for genre in game_info['categories']:
            info['Category: ' + genre['description']] = True
    
    return info

As previously mentioned, there are two APIs that will be queried to gather all of the desired information. These APIs seem to have a limit of 200 calls for every 5 minutes, so to account for this we will design special functions for both APIs that will wait until the response code 429 ("Too Many Requests") passes.

In [24]:
def store_query(app_id):
    global filters, key
    
    max_attempts = 11
    attempts = 0
    
    URL = f"http://store.steampowered.com/api/appdetails/?appids={app_id}"
    PARAMS = {'filters': filters, 'l': 'en', 'key': key}
    
    while attempts < max_attempts:
        re = requests.get(url=URL, params=PARAMS)

        if re.status_code != 429:
            break
            
        time.sleep(30)
        attempts += 1
        
    if attempts >= max_attempts:
        print(f"ERROR: Store query timeout on {app_id}")
        return None
    else:
        return re
    
def review_query(app_id):
    max_attempts = 11
    attempts = 0
    
    URL = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
    PARAMS = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
    
    while attempts < max_attempts:
        re = requests.get(url=URL, params=PARAMS)

        if re.status_code != 429:
            break
            
        time.sleep(30)
        attempts += 1
        
    if attempts >= max_attempts:
        print(f"ERROR: Review query timeout on {app_id}")
        return None
    else:
        return re

With all of that out of the way, we are able to collect the data.

In [25]:
def generate_df():
    global all_ids
    
    all_info = []
    for app_id in all_ids:
        # make main api query
        re = store_query(app_id)

        if re is not None and re.json()[str(app_id)]['success']:
            game_info = re.json()[str(app_id)]['data']
            info = fetch(game_info, categories)

            # make review api query
            re = review_query(app_id)
            if re is not None:
                summary = re.json()
                if 'query_summary' in summary.keys():
                    q_summary = summary['query_summary']

                    if 'total_positive' in q_summary:
                        info['total_positive'] = q_summary['total_positive']
                    else:
                        info['total_positive'] = np.NaN

                    if 'total_reviews' in q_summary:
                        info['total_reviews'] = q_summary['total_reviews']
                    else:
                        info['total_reviews'] = np.NaN
            
            # add the gathered information to the list
            all_info.append(info)

    df = pd.DataFrame.from_dict(all_info)
    return df

For practicality reasons, we collected the data beforehand and stored it locally as a pickle file so that we could easily load it back up without having to regather the data. This is because the API request limitations mean that data collection takes about 29 hours.

In [4]:
df = pd.read_pickle("games_df.pkl")
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,...,Genre: Web Publishing,Genre: Accounting,Genre: Video Production,Genre: Game Development,Genre: Photo Editing,Genre: Documentary,Genre: Episodic,Genre: Short,Genre: Tutorial,Genre: 360 Video
0,Counter-Strike,game,10,[Valve],[Valve],False,999.0,NaN,"Nov 1, 2000",88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Team Fortress Classic,game,20,[Valve],[Valve],False,499.0,NaN,"Apr 1, 1999",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Day of Defeat,game,30,[Valve],[Valve],False,499.0,NaN,"May 1, 2003",79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deathmatch Classic,game,40,[Valve],[Valve],False,499.0,NaN,"Jun 1, 2001",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Half-Life: Opposing Force,game,50,[Gearbox Software],[Valve],False,499.0,NaN,"Nov 1, 1999",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Processing

The dataframe now as all the information we want, but not all of it is in the desired format. First we replace NaN values from the genre or category columns with false. Values in the genres column are set to NaN if the app was not tagged as part of that genre/category, so this is a logical replacement value.

In [15]:
for col_name in df:
    if col_name.startswith('Genre: ') or col_name.startswith('Category: '):
        df[col_name].fillna(False, inplace=True)

Next we change the release dates to all be in the same format, the pandas datetime format. The parameter `errors='coerce'` means that if pandas cannot figure out what the date format means, then it will set the value to pd.NaT. This is reasonable because there are many apps and not all will use a recognizable format, but we do not want those few exceptions to bring the process to a complete stop. The parameter `infer_datetime_format=True` is used because there is no consistent date format for all Steam pages. The main reason the release date would not be in a format recognized by pandas would be if the date is given in a language other than English.

In [8]:
df['release_date date'] = df['release_date date'].apply(lambda x: pd.to_datetime(x, errors='coerce', infer_datetime_format=True))

Now we define a function to determine positive review percentage for a row. Gives NaN if missing either the total review count or total count of positive reviews.

In [9]:
def positive_perc(row):
    if not np.isnan(row['total_reviews']) and row['total_reviews'] != 0 and not np.isnan(row['total_positive']):
        return row['total_positive'] / row['total_reviews']
    else:
        return np.NaN

With the function defined, we can now add a positive review percentage column to the dataframe.

In [10]:
df['positive_percentage'] = df.apply(positive_perc, axis=1)

We can see that for free games, the price column tends to be NaN. It is reasonable to say that if a game is free, then its price should be 0. We will now define a function to do this for us.

In [11]:
def get_price(row):
    if np.isnan(row['price_overview initial']) and row['is_free']:
        return 0
    else:
        return row['price_overview initial']

We apply this function to the price_overview initial column, and rename the column to just price for the sake of brevity.

In [12]:
df['price_overview initial'] = df.apply(get_price, axis=1)
df.rename({'price_overview initial': 'price'}, axis='columns', inplace=True)

Seems reasonable to replace NaN achievements total with 0

In [13]:
df['achievements total'].fillna(0, inplace=True)

Now let's get an idea of what the tidied up dataframe looks like

In [14]:
df.head()

,name,type,steam_appid,developers,publishers,is_free,price,achievements total,release_date date,metacritic score,...,Genre: Accounting,Genre: Video Production,Genre: Game Development,Genre: Photo Editing,Genre: Documentary,Genre: Episodic,Genre: Short,Genre: Tutorial,Genre: 360 Video,positive_percentage
0,Counter-Strike,game,10,[Valve],[Valve],False,999.0,0.0,2000-11-01,88.0,...,False,False,False,False,False,False,False,False,False,0.974959
1,Team Fortress Classic,game,20,[Valve],[Valve],False,499.0,0.0,1999-04-01,NaN,...,False,False,False,False,False,False,False,False,False,0.858584
2,Day of Defeat,game,30,[Valve],[Valve],False,499.0,0.0,2003-05-01,79.0,...,False,False,False,False,False,False,False,False,False,0.900992
3,Deathmatch Classic,game,40,[Valve],[Valve],False,499.0,0.0,2001-06-01,NaN,...,False,False,False,False,False,False,False,False,False,0.820000
4,Half-Life: Opposing Force,game,50,[Gearbox Software],[Valve],False,499.0,0.0,1999-11-01,NaN,...,False,False,False,False,False,False,False,False,False,0.952906


## Exploratory Data Analysis